<a href="https://colab.research.google.com/github/nehadubey1205/NLP-college/blob/main/bullyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns

import re 
import nltk 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from nltk import sent_tokenize, word_tokenize
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer

from sklearn.model_selection import train_test_split
from time import time

import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

from google.colab import files
from matplotlib import pyplot as plt
import pandas as pd
import io
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv(r'/content/Bully_Sent_Emo_IEEEIC_6084.csv')

data = data.apply(lambda x: x.astype(str).str.lower())

data.head(2)

,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label
0,teri maa ki chut me shivsena ka lund gaya kya,teri maa ki chut me shivsena ka lund gaya kya,1,negative,anger
1,han muje pata hai teri maa ka ke gufa me jati thi,han muje pata hai teri maa ka ke gufa me jati thi,1,negative,others


In [3]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=9aae0bc09b95bcab835eb54994c1369b86ac1a3e70cea6962d26a247e099abf4
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [4]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 6.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=d3b82ffba0b58ae915f30d1609e383b47e64f3dacb02cd8e34f90ec16f2db0cb
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
 

In [5]:
from googletrans import Translator

In [8]:
translator = Translator()
translations = {}

In [9]:
data['translated_value'] = data['Processed_Tweets'].apply(lambda x: translator.translate(x, dest='en').text)

In [11]:
data.tail(2)

,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label,translated_value
6082,chal be bhadwe sanghi.. teri maa ki choot me g...,chal be bhadwe sanghi teri maa ki choot me gho...,1,negative,disgust,Let's go crazy with your mother's pussy.
6083,"इश्क़ में थे जब,\nवादा हमारा,\nदावा भी हमारा,\...",इश्क़ में थे जबवादा हमारादावा भी हमाराधोखा उनक...,0,neutral,sadness,"When we were in love, our claim was also our b..."


In [12]:
# Import required libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
first_column = data.pop('translated_value')
  
# insert column using insert(position,column_name,
# first_column) function
data.insert(0, 'translated_value', first_column)
  
#print()
#print("After Shifting column to first position")


In [18]:
display(data.tail(5))

,translated_value,Bully_Label
6079,dali sali kayo fhshdhdhdhd no need to follow m...,0
6080,tum rajput pehle mughalon ka bed hot krti thi ...,1
6081,"on the occasion of circumcision, tell me one t...",1
6082,let's go crazy with your mother's pussy.,1
6083,"when we were in love, our claim was also our b...",0


NOW WE WILL APLY BERT MODEL

In [20]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.5 MB/s eta 0:00:00


In [21]:
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
# Load the BERT model and tokenizer
model_class = ppb.BertModel
tokenizer_class = ppb.BertTokenizer
pretrained_weights = 'bert-base-multilingual-cased'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

data['translated_value'] = data['translated_value'].apply(lambda x: x.lower())
data['translated_value'] = data['translated_value'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
data.head(2)

NameError: ignored

In [2]:


# Pad the tokenized text to the same length
max_len = 0
for i in data['translated_value']:
    if len(i) > max_len:
        max_len = len(i)
padded = torch.zeros((len(data['translated_value']), max_len)).long()
for i, l in enumerate(data['translated_value']):
    padded[i][:len(l)] = torch.tensor(l)




NameError: ignored

In [28]:
# Split the dataset into training and testing sets
labels = data['Bully_Label']

In [1]:
input_ids = padded
attention_mask = input_ids != 0
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.2)
train_masks, test_masks, _, _ = train_test_split(attention_mask, input_ids, random_state=42, test_size=0.2)

NameError: ignored

In [ ]:



# Generate embeddings using the BERT model
with torch.no_grad():
    train_embeddings = model(train_inputs, attention_mask=train_masks)
    test_embeddings = model(test_inputs, attention_mask=test_masks)

# Train a logistic regression model
train_features = train_embeddings[0][:,0,:].numpy()
test_features = test_embeddings[0][:,0,:].numpy()
model = LogisticRegression()
model.fit(train_features, train_labels)

# Evaluate the model
y_pred = model.predict(test_features)
accuracy = accuracy_score(test_labels, y_pred)
print('Accuracy:', accuracy)

Note:till here again run and find the issue

In [14]:


# get the name of the column to move
col_name = 'translated_value'

# get the index of the column to move
col_index = data.columns.get_loc(col_name)

# remove the column from the DataFrame
col = data.pop(col_name)

# insert the column at the desired position
new_col_index = 1
data.insert(new_col_index, col_name, col)

# print the result
#print(data)


In [15]:
data.head(2)

,Bully_Label,translated_value
0,1,did shiv sena's dick go in your mother's pussy?
1,1,yes i know your mother used to go to the cave


In [13]:


# Load the dataset
#data = pd.read_csv('cyberbullying_dataset.csv')

# Remove irrelevant columns and clean the text
data = data.drop(['Tweets',	'Processed_Tweets',	'Sentiment_label',	'Emotion_label'], axis=1)
data['translated_value'] = data['translated_value'].apply(lambda x: x.lower())
data['translated_value'] = data['translated_value'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords.words('english'))]))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Vectorize the text using the TF-IDF algorithm
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


LookupError: ignored